In [1]:
!pip install transformers faiss-cpu torch scikit-learn

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
import faiss
import torch
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer

# Dados de exemplo
documents = [
    "A Torre Eiffel tem 324 metros de altura.",
    "A Estátua da Liberdade fica em Nova York.",
    "O Monte Everest é a montanha mais alta do mundo.",
]

# Vetorização dos documentos
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def embed(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.numpy()

document_embeddings = embed(documents)

# Indexação com FAISS
index = faiss.IndexFlatL2(document_embeddings.shape[1])
index.add(document_embeddings)

# Modelo Gerador
generator_tokenizer = AutoTokenizer.from_pretrained("t5-small")
generator_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Função de busca e geração
def rag(question, top_k=1):
    # Embedding da pergunta
    question_embedding = embed([question])
    # Recuperação dos documentos mais similares
    distances, indices = index.search(question_embedding, top_k)
    retrieved_docs = [documents[idx] for idx in indices[0]]
    # Concatenar documentos recuperados ao prompt
    input_text = " ".join(retrieved_docs) + " " + question
    inputs = generator_tokenizer.encode(input_text, return_tensors="pt", truncation=True)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    outputs = generator_model.generate(inputs, max_length=50, num_beams=2)
    answer = generator_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Exemplo de uso
question = "Onde fica a Estátua da Liberdade?"
print(rag(question))


Onde fica a Estátua da Liberdade fica em Nova York.


# Explicação do Código

## 1. Embeddings dos Documentos
- Utilizamos um modelo de embeddings para converter os documentos em vetores numéricos.

## 2. Indexação com FAISS
- Criamos um índice para busca eficiente de vetores similares.

## 3. Modelo Gerador
- Carregamos um modelo de geração de texto (T5 pequeno).

## 4. Função RAG
A função RAG realiza as seguintes etapas:
1. Recupera os documentos mais relevantes para a pergunta.
2. Combina os documentos recuperados com a pergunta.
3. Gera a resposta usando o modelo gerador.

## 5. Exemplo de Uso
- Perguntamos sobre a altura da Torre Eiffel e o modelo gera a resposta.